## Tensorflow v 1.15
### Basic classification

In [223]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [224]:
tf.version

<module 'tensorflow._api.v1.version' from 'C:\\Users\\user\\Anaconda3\\envs\\old python\\lib\\site-packages\\tensorflow_core\\_api\\v1\\version\\__init__.py'>

## Titanic dataset

Datasets EDA are in the other notebook

In [225]:
data = pd.read_csv('Titanic.csv', index_col=0)
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [226]:
data.drop(['Name', 'Ticket', "Cabin", 'Embarked'], axis=1, inplace=True)

#### Titanic dataset columns

- Survived (TARGET): Flag, true if passanger survived accident
- PClass: 1 for 1st class, 2 for 2nd, 3 for 3rd (cat feature)
- Sex (cat feature)
- Age
- SibSp amount of siblings on the board
- Parch amount of parents/children on the board
- Fare 


In [227]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
1,0,3,male,22.0,1,0,7.2500
2,1,1,female,38.0,1,0,71.2833
3,1,3,female,26.0,0,0,7.9250
4,1,1,female,35.0,1,0,53.1000
5,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
887,0,2,male,27.0,0,0,13.0000
888,1,1,female,19.0,0,0,30.0000
889,0,3,female,NaN,1,2,23.4500


In [228]:
from sklearn.model_selection import train_test_split

cat_cols = ['Pclass', 'Sex']
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
y = data['Survived']
X = data.drop('Survived', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, stratify=y)

In [229]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_trans = Pipeline(steps=[
    ('imputer', KNNImputer()),
    
])
cat_trans = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
transformers =[
    ('num', num_trans, num_cols),
    ('cat', cat_trans, cat_cols),
], remainder='drop')

In [230]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()
X_train = pd.DataFrame(preprocessor.fit_transform(X_train))
X_test = pd.DataFrame(preprocessor.transform(X_test))

In [231]:
y_train = pd.DataFrame(y_train)
y_train['Not'] = y_train['Survived'] == 0

In [232]:
y_test = pd.DataFrame(y_test)
y_test['Not'] = y_test['Survived'] == 0

#### Softmax regression for classification for 2 classes

In [233]:
tf.reset_default_graph()

num_classes = y_train.shape[1]
learning_rate = 0.01
n_epochs = 30
batch_size = 50

X = tf.placeholder(tf.float32, [batch_size, X_train.shape[1]], name='data') 
Y = tf.placeholder(tf.int32, [batch_size, 2], name='label')


w = tf.get_variable(name='weights',shape=(X_train.shape[1], num_classes), initializer=tf.random_normal_initializer(), )
b = tf.get_variable(name = 'bias', shape=(1, num_classes), initializer=tf.zeros_initializer())


logits = tf.matmul(X, w) + b 


entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy)


optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    

    for i in range(n_epochs): 
        total_loss = 0
        n_batches = X_train.shape[0] // batch_size
        for j in range(n_batches-1):
            X_batch, y_batch = X_train.iloc[j*batch_size:(j+1)*batch_size], y_train.iloc[j*batch_size:(j+1)*batch_size]
            _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y: y_batch}) 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/(n_batches-1)))


    total_correct_preds = 0
    n_batches = X_test.shape[0] // batch_size
    for j in range(n_batches - 1):
        X_batch, Y_batch = X_test.iloc[j*batch_size:(j+1)*batch_size], y_test.iloc[j*batch_size:(j+1)*batch_size]
        accuracy_batch = sess.run(accuracy, {X: X_batch, Y:Y_batch})
        total_correct_preds += accuracy_batch  
    print('Accuracy {0}'.format(total_correct_preds/((n_batches - 1) * batch_size)))

Average loss epoch 0: 12.875968138376871
Average loss epoch 1: 11.363237599531809
Average loss epoch 2: 10.066785037517548
Average loss epoch 3: 8.808952490488688
Average loss epoch 4: 7.530684947967529
Average loss epoch 5: 6.23762987057368
Average loss epoch 6: 4.94433789451917
Average loss epoch 7: 3.6659236947695413
Average loss epoch 8: 2.4464094837506614
Average loss epoch 9: 1.4588999350865681
Average loss epoch 10: 0.9650745391845703
Average loss epoch 11: 0.9343263804912567
Average loss epoch 12: 0.8715998604893684
Average loss epoch 13: 0.8115653470158577
Average loss epoch 14: 0.770466518898805
Average loss epoch 15: 0.7383232489228249
Average loss epoch 16: 0.7103373085459074
Average loss epoch 17: 0.6828954343994459
Average loss epoch 18: 0.6587137654423714
Average loss epoch 19: 0.6372869039575259
Average loss epoch 20: 0.6180267756183943
Average loss epoch 21: 0.6005582759777705
Average loss epoch 22: 0.5847223202387491
Average loss epoch 23: 0.5704403867324194
Average l

In [234]:
titanic_X_train, titanic_y_train, titanic_X_test, titanic_y_test = X_train, y_train, X_test, y_test

### Thyroid Disease dataset

EDA and preprocessing are taken from HT3

In [235]:
data = pd.read_csv('dataset_57_hypothyroid.csv')
data = data.drop(['TBG_measured','TBG'], axis=1)
for i in ['TSH','T3','TT4','FTI','T4U']:
    data[i] = data[i].apply(lambda x: 0 if x == '?' else x)
data['sex'] = data['sex'].apply(lambda x: 'U' if x == '?' else x) 
data['age'] = data['age'].replace('?', np.median(pd.to_numeric(data.query('sex == "U" and age != "?"')['age']) ))
num_cols = [col for col in data.columns if data[col].dtype in ['int64', 'float64']]

In [236]:
bool_cols = [col for col in data.columns if data[col].nunique() == 2 and data[col][0] in ['t', 'f']]
for col in bool_cols:
    data[col] = data[col] == 't'

In [237]:
data.drop('hypopituitary', axis=1, inplace=True)
data.drop_duplicates(inplace=True)

In [238]:
TARGET = 'Class'
SEED = 13

In [239]:
data = data[data[TARGET] != 'secondary_hypothyroid']
X = data.drop(TARGET, axis=1)
y = data[TARGET]
cat_cols = [col for col in X.columns if X[col].dtype =='object']
for col in cat_cols:
    try:
        X[col] = pd.to_numeric(X[col])
    except:
        pass
cat_cols = [col for col in X.columns if X[col].dtype =='object']
num_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
bool_cols = [col for col in X.columns if X[col].dtype == 'bool']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
# X_train[cat_cols] = X_train[cat_cols].apply(lambda x: str(x))
# X_test[cat_cols] = X_test[cat_cols].apply(str)

In [240]:
X[cat_cols]

,sex,referral_source
0,F,SVHC
1,F,other
2,M,other
3,F,other
4,F,SVI
...,...,...
3767,F,other
3768,F,SVI
3769,F,other
3770,M,SVI


In [241]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=SEED)

In [242]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

num_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler())
    
])
cat_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

bool_trans = Pipeline(steps=[
    ('encoder', OrdinalEncoder())
])


preprocessor = ColumnTransformer(
transformers =[
    ('num', num_trans, num_cols),
    ('cat', cat_trans, cat_cols),
    ('bool', bool_trans, bool_cols)
], remainder='drop')

In [243]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train))
X_test = pd.DataFrame(preprocessor.transform(X_test))

In [244]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1.074125,-0.127238,0.425800,0.399228,0.214244,0.466885,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,1.220544,0.050534,-0.399673,-0.359760,-0.507201,0.444438,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2,0.390837,-0.096321,0.425800,-0.261826,0.154123,-0.094300,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
3,-0.390065,-0.115644,0.517520,-0.016991,0.875568,-0.341222,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
4,-0.390065,0.038940,0.700958,-1.045298,-2.731657,-2.271699,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,-0.292452,-0.146561,1.067835,0.962348,1.176170,0.197516,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2777,0.049192,-0.156223,0.334081,-0.016991,0.063943,0.219964,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2778,1.269351,-0.030623,0.242362,0.301294,0.424665,0.197516,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2779,0.878900,-0.175932,0.792677,1.452018,0.605026,1.050518,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [245]:
_y_train = pd.DataFrame()
_y_test = pd.DataFrame()
for j in range(pd.Series(y_train).nunique()):
    _y_train[j] = y_train == j
    _y_test[j] = y_test == j

In [246]:
_y_test

,0,1,2
0,False,True,False
1,False,True,False
2,True,False,False
3,False,True,False
4,False,True,False
...,...,...,...
923,True,False,False
924,False,True,False
925,True,False,False
926,False,True,False


In [247]:
tf.reset_default_graph()

num_classes = _y_train.shape[1]
learning_rate = 0.01
n_epochs = 30
batch_size = 100

X = tf.placeholder(tf.float32, [batch_size, X_train.shape[1]], name='data') 
Y = tf.placeholder(tf.int32, [batch_size, num_classes], name='label')


w = tf.get_variable(name='weights',shape=(X_train.shape[1], num_classes), initializer=tf.random_normal_initializer(), )
b = tf.get_variable(name = 'bias', shape=(1, num_classes), initializer=tf.zeros_initializer())


logits = tf.matmul(X, w) + b 


entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy)


optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    

    for i in range(n_epochs): 
        total_loss = 0
        n_batches = X_train.shape[0] // batch_size
        for j in range(n_batches-1):
            X_batch, y_batch = X_train.iloc[j*batch_size:(j+1)*batch_size], _y_train.iloc[j*batch_size:(j+1)*batch_size]
            _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y: y_batch}) 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/(n_batches-1)))


    total_correct_preds = 0
    n_batches = X_test.shape[0] // batch_size
    for j in range(n_batches - 1):
        X_batch, Y_batch = X_test.iloc[j*batch_size:(j+1)*batch_size], _y_test.iloc[j*batch_size:(j+1)*batch_size]
        accuracy_batch = sess.run(accuracy, {X: X_batch, Y:Y_batch})
        total_correct_preds += accuracy_batch  
    print('Accuracy {0}'.format(total_correct_preds/((n_batches - 1) * batch_size)))

Average loss epoch 0: 1.8245101754481976
Average loss epoch 1: 0.590573305120835
Average loss epoch 2: 0.4009706767705771
Average loss epoch 3: 0.3338426047792801
Average loss epoch 4: 0.2892147468832823
Average loss epoch 5: 0.25691771793823975
Average loss epoch 6: 0.23684525174590257
Average loss epoch 7: 0.22484114250311485
Average loss epoch 8: 0.2162963143334939
Average loss epoch 9: 0.20970032879939446
Average loss epoch 10: 0.20435771947869888
Average loss epoch 11: 0.19988287899356622
Average loss epoch 12: 0.19603668812375802
Average loss epoch 13: 0.1926637446651092
Average loss epoch 14: 0.18965935133970702
Average loss epoch 15: 0.18695049360394478
Average loss epoch 16: 0.1844846448646142
Average loss epoch 17: 0.18222283571958542
Average loss epoch 18: 0.18013537302613258
Average loss epoch 19: 0.1781990576822024
Average loss epoch 20: 0.17639531682317072
Average loss epoch 21: 0.1747090111558254
Average loss epoch 22: 0.17312751051325065
Average loss epoch 23: 0.1716402

In [248]:
TD_X_train, TD_y_train, TD_X_test, TD_y_test = X_train, _y_train, X_test, _y_test

## Keras

In [249]:
import keras
from keras.models import Sequential
from keras.layers import Dense


### Titanic

In [266]:
model = Sequential()
model.add(Dense(titanic_y_train.shape[1], activation='softmax', input_shape=(titanic_X_train.shape[1],)))
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(titanic_X_train, titanic_y_train,
          batch_size=50, epochs=30,
          validation_data = (titanic_X_test, titanic_y_test))

Train on 668 samples, validate on 223 samples
Epoch 1/30
668/668 [==============================] - 0s 345us/step - loss: 15.4650 - accuracy: 0.3832 - val_loss: 16.8197 - val_accuracy: 0.3857
Epoch 2/30
668/668 [==============================] - 0s 43us/step - loss: 14.5461 - accuracy: 0.3832 - val_loss: 15.7871 - val_accuracy: 0.3857
Epoch 3/30
668/668 [==============================] - 0s 60us/step - loss: 13.5887 - accuracy: 0.3832 - val_loss: 14.7749 - val_accuracy: 0.3857
Epoch 4/30
668/668 [==============================] - 0s 52us/step - loss: 12.6730 - accuracy: 0.3832 - val_loss: 13.7488 - val_accuracy: 0.3857
Epoch 5/30
668/668 [==============================] - 0s 54us/step - loss: 11.7301 - accuracy: 0.3832 - val_loss: 12.7353 - val_accuracy: 0.3857
Epoch 6/30
668/668 [==============================] - 0s 55us/step - loss: 10.7837 - accuracy: 0.3832 - val_loss: 11.7342 - val_accuracy: 0.3857
Epoch 7/30
668/668 [==============================] - 0s 52us/step - loss: 9.8611 -

In [273]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(titanic_X_train.shape[1],)))
model.add(Dense(titanic_y_train.shape[1], activation='softmax'))
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(titanic_X_train, titanic_y_train,
          batch_size=50, epochs=30,
          validation_data = (titanic_X_test, titanic_y_test))

Train on 668 samples, validate on 223 samples
Epoch 1/30
668/668 [==============================] - 0s 473us/step - loss: 2.5525 - accuracy: 0.6168 - val_loss: 1.9713 - val_accuracy: 0.6143
Epoch 2/30
668/668 [==============================] - 0s 47us/step - loss: 1.5061 - accuracy: 0.6123 - val_loss: 1.1435 - val_accuracy: 0.5830
Epoch 3/30
668/668 [==============================] - 0s 53us/step - loss: 0.9403 - accuracy: 0.6048 - val_loss: 0.9064 - val_accuracy: 0.6054
Epoch 4/30
668/668 [==============================] - 0s 51us/step - loss: 0.8105 - accuracy: 0.6287 - val_loss: 0.8600 - val_accuracy: 0.6009
Epoch 5/30
668/668 [==============================] - 0s 46us/step - loss: 0.7579 - accuracy: 0.6602 - val_loss: 0.8082 - val_accuracy: 0.5919
Epoch 6/30
668/668 [==============================] - 0s 53us/step - loss: 0.7138 - accuracy: 0.6766 - val_loss: 0.7741 - val_accuracy: 0.5919
Epoch 7/30
668/668 [==============================] - 0s 51us/step - loss: 0.6823 - accuracy: 0

### Thyroid Disease

In [267]:
model = Sequential()
model.add(Dense(TD_y_train.shape[1], activation='softmax', input_shape=(TD_X_train.shape[1],)))
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.fit(TD_X_train, TD_y_train,
          batch_size=100, epochs=20,
          validation_data = (TD_X_test, TD_y_test))

Train on 2781 samples, validate on 928 samples
Epoch 1/20
2781/2781 [==============================] - 0s 90us/step - loss: 1.0135 - accuracy: 0.5038 - val_loss: 0.8929 - val_accuracy: 0.6401
Epoch 2/20
2781/2781 [==============================] - 0s 19us/step - loss: 0.8021 - accuracy: 0.7033 - val_loss: 0.7149 - val_accuracy: 0.7468
Epoch 3/20
2781/2781 [==============================] - 0s 20us/step - loss: 0.6499 - accuracy: 0.7979 - val_loss: 0.5913 - val_accuracy: 0.8459
Epoch 4/20
2781/2781 [==============================] - 0s 22us/step - loss: 0.5455 - accuracy: 0.8749 - val_loss: 0.5072 - val_accuracy: 0.8782
Epoch 5/20
2781/2781 [==============================] - 0s 18us/step - loss: 0.4750 - accuracy: 0.8986 - val_loss: 0.4488 - val_accuracy: 0.8944
Epoch 6/20
2781/2781 [==============================] - 0s 20us/step - loss: 0.4257 - accuracy: 0.9108 - val_loss: 0.4093 - val_accuracy: 0.9149
Epoch 7/20
2781/2781 [==============================] - 0s 20us/step - loss: 0.3919

### Boston dataset

In [278]:
from sklearn.datasets import load_boston

boston = load_boston()
boston_data = pd.DataFrame(boston.data, columns=boston.feature_names)

In [280]:
boston_target = pd.Series(boston.target)

In [281]:
cols_with_missing = [col for col in boston_data.columns
                     if boston_data[col].isnull().any()]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston_data, boston_target, random_state=SEED)

In [284]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1.074125,-0.127238,0.425800,0.399228,0.214244,0.466885,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1,1.220544,0.050534,-0.399673,-0.359760,-0.507201,0.444438,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2,0.390837,-0.096321,0.425800,-0.261826,0.154123,-0.094300,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
3,-0.390065,-0.115644,0.517520,-0.016991,0.875568,-0.341222,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
4,-0.390065,0.038940,0.700958,-1.045298,-2.731657,-2.271699,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,-0.292452,-0.146561,1.067835,0.962348,1.176170,0.197516,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2777,0.049192,-0.156223,0.334081,-0.016991,0.063943,0.219964,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2778,1.269351,-0.030623,0.242362,0.301294,0.424665,0.197516,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2779,0.878900,-0.175932,0.792677,1.452018,0.605026,1.050518,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [289]:
model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu, input_shape=[X_train.shape[1]]),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

mse = tf.keras.losses.MeanSquaredError()
model.compile('adam', mse, metrics=['MSE'])

model.fit(X_train, y_train,
          batch_size=100, epochs=15,
          validation_data = (X_test, y_test))

Train on 2781 samples, validate on 928 samples
Epoch 1/15
2781/2781 [==============================] - 0s 135us/step - loss: 0.1647 - MSE: 0.1647 - val_loss: 0.0902 - val_MSE: 0.0902
Epoch 2/15
2781/2781 [==============================] - 0s 20us/step - loss: 0.0760 - MSE: 0.0760 - val_loss: 0.0687 - val_MSE: 0.0687
Epoch 3/15
2781/2781 [==============================] - 0s 18us/step - loss: 0.0650 - MSE: 0.0650 - val_loss: 0.0639 - val_MSE: 0.0639
Epoch 4/15
2781/2781 [==============================] - 0s 19us/step - loss: 0.0603 - MSE: 0.0603 - val_loss: 0.0609 - val_MSE: 0.0609
Epoch 5/15
2781/2781 [==============================] - 0s 18us/step - loss: 0.0569 - MSE: 0.0569 - val_loss: 0.0583 - val_MSE: 0.0583
Epoch 6/15
2781/2781 [==============================] - 0s 19us/step - loss: 0.0541 - MSE: 0.0541 - val_loss: 0.0566 - val_MSE: 0.0566
Epoch 7/15
2781/2781 [==============================] - 0s 17us/step - loss: 0.0519 - MSE: 0.0519 - val_loss: 0.0552 - val_MSE: 0.0552
Epoch 8